In [1]:
# !pip install opencv-python

In [2]:
import tensorflow as tf # this is to import tensorflow
import json # this is to import json
import pandas as pd # this is to import pandas
import numpy as np # this is to import numpy
from skimage import io, transform # this is to import skimage
from sklearn.model_selection import train_test_split # this is just for train test split lol
import cv2 # compooter vision
from tensorflow.keras import datasets, layers, models, losses, Model # this imports things
from tqdm.notebook import tqdm # this is literally just a progress bar
import os # operating system things
import glob
from PIL import Image # this is to resize images

In [3]:
# Read the json
with open("amogus2.json", 'r') as f:
    data_df = pd.json_normalize([i for i in json.loads(f.read()) if i is not None])
data_df = data_df.set_index("id")
data_df

,url_256,url_1024,country,lat,lng,location_id
id,,,,,,
1509662729539070,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,null,0.000000e+00,0.000000e+00,090180
1587956288269248,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,null,0.000000e+00,0.000000e+00,090180
1414341782427856,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,null,0.000000e+00,0.000000e+00,090180
1504003806771142,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,null,5.404263e-08,-9.539572e-07,090170
1502346620201016,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,null,0.000000e+00,0.000000e+00,090180
...,...,...,...,...,...,...
4204140206347239,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,Thailand,9.455834e+00,9.920649e+01,090270
1405412416480572,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,Thailand,9.389187e+00,9.916939e+01,090270
3963740523712531,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,Thailand,9.670108e+00,9.911407e+01,090270


In [4]:
value_counts = data_df['location_id'].value_counts()

# create a boolean mask to select the rows where the value occurs >= 10 times
mask = data_df['location_id'].isin(value_counts[value_counts >= 10].index)

# apply the boolean mask to the dataframe to filter out the rows
df_filtered = data_df[mask]
df_filtered

,url_256,url_1024,country,lat,lng,location_id
id,,,,,,
1509662729539070,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,null,0.000000e+00,0.000000e+00,090180
1587956288269248,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,null,0.000000e+00,0.000000e+00,090180
1414341782427856,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,null,0.000000e+00,0.000000e+00,090180
1504003806771142,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,null,5.404263e-08,-9.539572e-07,090170
1502346620201016,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,null,0.000000e+00,0.000000e+00,090180
...,...,...,...,...,...,...
4204140206347239,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,Thailand,9.455834e+00,9.920649e+01,090270
1405412416480572,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,Thailand,9.389187e+00,9.916939e+01,090270
3963740523712531,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,Thailand,9.670108e+00,9.911407e+01,090270


In [5]:
images = {file: cv2.imread(file) for file in tqdm(glob.glob("img2/*.jpg"))}

  0%|          | 0/76439 [00:00<?, ?it/s]

In [6]:
img_series = pd.Series(images)
img_series = img_series[img_series.notna()].copy()
img_series = img_series[[i.shape==(192, 256, 3) for i in tqdm(img_series)]].copy() # change to reshape
# img_series = img_series.apply(lambda x: reshape(x))
img_series = img_series.rename(lambda str: str[5:-4]) # convert file name e.g. "img2\1000148914149750.jpg"->1000148914149750
img_series

  0%|          | 0/76433 [00:00<?, ?it/s]

1000148914149750    [[[200, 195, 180], [201, 196, 181], [202, 197,...
1000165370829738    [[[230, 214, 177], [229, 213, 176], [228, 212,...
1000308077173457    [[[255, 254, 255], [255, 254, 255], [255, 255,...
1000346054206532    [[[197, 208, 205], [244, 252, 251], [241, 245,...
1000390084038009    [[[61, 63, 71], [65, 61, 66], [70, 59, 55], [9...
                                          ...                        
999823017360399     [[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...
999914380882771     [[[233, 210, 188], [233, 210, 188], [233, 210,...
999922524079739     [[[56, 43, 59], [64, 53, 69], [80, 69, 85], [8...
999991773868952     [[[176, 167, 163], [176, 167, 163], [176, 167,...
999995657071410     [[[199, 188, 180], [199, 188, 180], [200, 189,...
Length: 76409, dtype: object

In [7]:
data_df["img"] = img_series
data_df = data_df.dropna()
data_df

,url_256,url_1024,country,lat,lng,location_id,img
id,,,,,,,
1509662729539070,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,null,0.000000e+00,0.000000e+00,090180,"[[[144, 149, 158], [144, 149, 158], [144, 149,..."
1587956288269248,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,null,0.000000e+00,0.000000e+00,090180,"[[[255, 237, 209], [255, 237, 208], [254, 236,..."
1414341782427856,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,null,0.000000e+00,0.000000e+00,090180,"[[[109, 113, 88], [255, 255, 238], [174, 175, ..."
1504003806771142,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,null,5.404263e-08,-9.539572e-07,090170,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
1502346620201016,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,null,0.000000e+00,0.000000e+00,090180,"[[[228, 233, 236], [218, 223, 226], [217, 224,..."
...,...,...,...,...,...,...,...
4204140206347239,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,Thailand,9.455834e+00,9.920649e+01,090270,"[[[190, 140, 82], [190, 140, 82], [191, 141, 8..."
1405412416480572,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,Thailand,9.389187e+00,9.916939e+01,090270,"[[[151, 147, 146], [151, 147, 146], [151, 147,..."
3963740523712531,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,Thailand,9.670108e+00,9.911407e+01,090270,"[[[147, 142, 151], [146, 141, 150], [147, 142,..."


In [8]:
country_df = data_df[data_df["country"] != "null"]
country_df

,url_256,url_1024,country,lat,lng,location_id,img
id,,,,,,,
323130512496076,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,Brazil,-0.648993,-47.681656,080130,"[[[215, 182, 137], [215, 182, 137], [215, 182,..."
477735123552105,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,Brazil,-0.646406,-47.680764,080130,"[[[176, 156, 115], [176, 157, 114], [179, 157,..."
973070119934407,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,Brazil,-0.649970,-47.682089,080130,"[[[196, 163, 117], [196, 163, 117], [196, 163,..."
508469686990915,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,Brazil,-0.644159,-47.679022,080130,"[[[221, 215, 208], [224, 218, 211], [224, 218,..."
287886579730767,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,Brazil,-0.645092,-47.679848,080130,"[[[250, 247, 242], [246, 243, 238], [240, 237,..."
...,...,...,...,...,...,...,...
4204140206347239,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,Thailand,9.455834,99.206494,090270,"[[[190, 140, 82], [190, 140, 82], [191, 141, 8..."
1405412416480572,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,Thailand,9.389187,99.169389,090270,"[[[151, 147, 146], [151, 147, 146], [151, 147,..."
3963740523712531,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,https://scontent.fsin14-2.fna.fbcdn.net/m1/v/t...,Thailand,9.670108,99.114070,090270,"[[[147, 142, 151], [146, 141, 150], [147, 142,..."


In [9]:
country_df.groupby(['country']).count().sort_values(by='img')

,url_256,url_1024,lat,lng,location_id,img
country,,,,,,
Ukraine,1,1,1,1,1,1
St. Lucia,1,1,1,1,1,1
eSwatini,2,2,2,2,2,2
Montenegro,2,2,2,2,2,2
Singapore,2,2,2,2,2,2
...,...,...,...,...,...,...
Canada,3100,3100,3100,3100,3100,3100
Brazil,4026,4026,4026,4026,4026,4026
Australia,4799,4799,4799,4799,4799,4799


In [10]:
wts = dict(1 / country_df.groupby(['country']).count().reset_index()["img"])
wts

{0: 0.012345679012345678,
 1: 0.02631578947368421,
 2: 0.0020408163265306124,
 3: 0.25,
 4: 0.004201680672268907,
 5: 0.16666666666666666,
 6: 0.025,
 7: 0.1,
 8: 0.0005685048322910744,
 9: 0.03333333333333333,
 10: 0.5,
 11: 0.00020837674515524068,
 12: 0.010752688172043012,
 13: 0.008,
 14: 0.1,
 15: 0.004739336492890996,
 16: 0.1,
 17: 0.00558659217877095,
 18: 0.01639344262295082,
 19: 0.023255813953488372,
 20: 0.023255813953488372,
 21: 0.1,
 22: 0.03333333333333333,
 23: 0.0017152658662092624,
 24: 0.024390243902439025,
 25: 0.002531645569620253,
 26: 0.00024838549428713363,
 27: 0.1,
 28: 0.029411764705882353,
 29: 0.007874015748031496,
 30: 0.00847457627118644,
 31: 0.09090909090909091,
 32: 0.009259259259259259,
 33: 0.0048543689320388345,
 34: 0.0003225806451612903,
 35: 0.011111111111111112,
 36: 0.1111111111111111,
 37: 0.01098901098901099,
 38: 0.011764705882352941,
 39: 0.0014367816091954023,
 40: 0.00043554006968641115,
 41: 0.0019569471624266144,
 42: 0.1,
 43: 0.33333

In [11]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='ignore')
enc_country_df = pd.DataFrame(encoder.fit_transform(country_df[['country']]).toarray())
enc_country_df

,0,1,2,3,4,5,6,7,8,9,...,205,206,207,208,209,210,211,212,213,214
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
X_train_index, X_test_index, y_train, y_test = train_test_split(list(country_df.index), enc_country_df, test_size=0.20)

In [13]:
X_train = np.stack(img_series[X_train_index].values)
X_test = np.stack(img_series[X_test_index].values)

In [14]:
resnet_model = tf.keras.applications.ResNet152(weights = 'imagenet', include_top = False, input_shape = (192,256,3))
unfreeze_layers = 5
for layer in resnet_model.layers[:-unfreeze_layers]:
    layer.trainable = False

In [15]:
model = models.Sequential()
#model.add(layers.Lambda(lambda image: tf.image.resize(image, (128, 256))))
model.add(resnet_model)
model.add(layers.Flatten())
model.add(layers.Dense(1000, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())
model.add(layers.Dense(len(enc_country_df.columns), activation="softmax"))

In [16]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=64, epochs=5, class_weight=wts, validation_data=(X_test[:100], y_test[:100]))

Epoch 1/5
938/938 [==============================] - 10940s 12s/step - loss: 1.3078e-05 - accuracy: 0.0154 - val_loss: 0.0046 - val_accuracy: 0.0200
Epoch 2/5
938/938 [==============================] - 10166s 11s/step - loss: 1.2256e-05 - accuracy: 0.0353 - val_loss: 0.0046 - val_accuracy: 0.0200
Epoch 3/5
228/938 [======>.......................] - ETA: 2:09:45 - loss: 1.1474e-05 - accuracy: 0.0420

In [ ]:
from collections import Counter
Counter([i.shape for i in tqdm(img_series)])

In [ ]:
theimg = io.imread("random_images/boarding.jpg") #country_df['image'][380]
reshaped = reshape()
io.imshow(reshaped)

In [ ]:
p = model.predict(reshaped[None, ...])
argmax = p[0].argmax()
encoder.categories_[0][argmax]

In [ ]:
io.imshow(country_df['image'][380])